In [12]:
%load_ext pycodestyle_magic
%pycodestyle_off 

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dataset
import torchvision.transforms as transforms

In [13]:
VGG16 = [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ]
class VGG_net(nn.Module):
    def __init__(self,in_channel=3,num_classes=1000):
        super(VGG_net,self).__init__()
        self.in_channel = in_channel
        self.conv_layer = self.create_conv_layer(VGG16)
        
        self.fcs = nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096,num_classes)
        )
    def forward(self,x):
        x=self.conv_layer(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fcs(x)
        return x
    def create_conv_layer(self,architecture):
        layer = []
        in_channel = self.in_channel
        
        for x in architecture:
            if type(x) == int:
                out_channel = x
                layer += [nn.Conv2d(in_channels=in_channel,out_channels=out_channel,
                                   kernel_size=(3,3),stride=(1,1),
                                    padding=(1,1)),
                         nn.BatchNorm2d(x),
                         nn.ReLU()]
                in_channel=x
            elif x=='M':
                layer += [nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))]
        return nn.Sequential(*layer)
model = VGG_net(in_channel=3,num_classes=1000)
x=torch.randn(1,3,224,224)
print(model(x).shape)

torch.Size([1, 1000])
